These produce a final .json file that can be used as an input in chemiscope to produce the final figures of the manuscript. 
The file can be already found in the downloaded folder.
However, if you want to run the notebook yourself, a new chemiscope file will be generated, along with new pcovr figures. 

In [ ]:
from data import DATA_3DCD, DATA_MP

In [2]:
%run ./modules.ipynb

sns.set(style="white", palette="muted", color_codes=True)
my_c = colors.ListedColormap(["mediumblue", "red"])
c = ["mediumblue", "red"]
sns.set_palette(sns.color_palette(c))

n_PC = 5  # number of principal components
mixing = 0.5  # beta parameter for pcvor

3DCD data analysis

In [3]:
npzfile = np.load(DATA_3DCD.soap, allow_pickle=True)
my_frames = npzfile["my_frames"]
# idx = npzfile["idx"]
magic = npzfile["magic"]
X_raw = npzfile["X_raw"]
coeff = npzfile["coeff"]

magic = [int(i) for i in magic]
magic = np.array(magic).reshape(-1, 1)
y_magic = np.copy(magic.reshape(-1, 1).ravel())

In [4]:
X = np.copy(X_raw)
y_magic = np.copy(magic.reshape(-1, 1))

if os.path.exists('train_indices.npy'):
    i_train, i_test = np.load('train_indices.npy'), np.load('test_indices.npy')
    X_train, X_test = X[i_train], X[i_test]
    y_train, y_test = y_magic[i_train], y_magic[i_test]
else:
    i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
        np.arange(X.shape[0]), X, y_magic, train_size=0.9
    )
    np.save('train_indices.npy', i_train)
    np.save('test_indices.npy', i_test)
    
x_scaler = StandardFlexibleScaler(column_wise=False).fit(X)
# Center total dataset
X = x_scaler.transform(X)
# Center training data
X_train = x_scaler.transform(X_train)
# Center training data
X_test = x_scaler.transform(X_test)

In [5]:
import pickle
if os.path.exists('random_forest.sav'):
    clf = pickle.load(open('random_forest.sav', 'rb'))
else:
    clf = RandomForestClassifier(n_estimators=100, random_state=2)
    clf.fit(X_train, y_train.ravel())
    pickle.dump(clf, open('random_forest.sav', 'wb'))
print("Accuracy on test set:" + str(clf.score(X_test, y_test.ravel())))

Accuracy on test set:0.822564487853744


In [21]:
clf.score

False

In [6]:
print("DONE")

DONE


In [11]:
from sklearn.svm import LinearSVC
if os.path.exists('linearsvc.sav'):
    svc = pickle.load(open('linearsvc.sav', 'rb'))
else:
    svc = LinearSVC(C=100, fit_intercept=False)
    svc.fit(X_train, y_magic[i_train])
    pickle.dump(svc, open('linearsvc.sav', 'wb'))

In [12]:
svc.score(X_test, y_magic[i_test]), svc.score(X_train, y_magic[i_train])

(0.6699223641372402, 0.6916847553848723)

In [15]:
from sklearn.inspection import DecisionBoundaryDisplay
fig, ax = plt.subplots(1)

t0 = np.linspace(X[:,0].min(), X[:,0].max(), 20)
t1 = np.linspace(X[:,1].min(), X[:,1].max(), 20)
xx0, xx1 = np.meshgrid(t0, t1)

In [ ]:
p_random_for = clf.predict_proba(X)

y_tot = p_random_for[:, 1].reshape(-1, 1)
i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
    np.arange(X.shape[0]), X, y_tot, train_size=0.8
)

y_scaler = StandardFlexibleScaler(column_wise=True).fit(y_tot)

Y = y_scaler.transform(y_tot)
y_train = y_scaler.transform(y_train)
y_test = y_scaler.transform(y_test)

pcovr = PCovR(mixing=mixing, n_components=n_PC)
pcovr.fit(X, Y)
T = pcovr.transform(X)
yp = pcovr.predict(X)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.scatter(T[:, 0], T[:, 1], s=50, alpha=0.3, c=p_random_for[:, 1], cmap="bwr")
ax1.set_xlabel(r"$PCov_1$")
ax1.set_ylabel(r"$PCov_2$")

ax2.scatter(Y, yp, s=50, alpha=0.5, c=np.abs(Y - yp), cmap="bone_r")
ax2.plot(ax2.get_xlim(), ax2.get_xlim(), "r--")
ax2.set_xlabel(r"y")
ax2.set_ylabel(r"predicted y")
sns.despine(left=True, bottom=True)

In [ ]:
fn = np.zeros(y_tot.shape, dtype=int)
fp = np.zeros(y_tot.shape, dtype=int)
fn[np.logical_and(y_tot<0.5, magic==1)] = 1.0
fp[np.logical_and(y_tot>=0.5, magic==0)] = 1.0

In [ ]:
import chemiscope
widget = chemiscope.show(my_frames, 
                         properties={"p_magic": y_tot, 
                                     "magic": magic, 
                                     "pcovr": T, 
                                     'fp':fp, 'fn':fn
                                    })

In [ ]:
widget.save('3dcd.json')

In [ ]:
my_frames[5086].wrap()
view(my_frames[5086])

MP data analysis

In [ ]:
if os.path.exists("../r4data"):
    npzfile = np.load(DATA_MP.soap_red, allow_pickle=True)
    my_frames = npzfile["my_frames"]
    idx = npzfile["idx"]
    magic = npzfile["magic"]
    X_raw = npzfile["X_raw"]
    coeff = npzfile["coeff"]
    magic = [int(i) for i in magic]
    magic = np.array(magic).reshape(-1, 1)
    y_magic = np.copy(magic.reshape(-1, 1).ravel())

    comp_classif(X_raw, y_magic)  # prints out test set accuracy on all models

    X = np.copy(X_raw)
    y_magic = np.copy(magic.reshape(-1, 1))

    i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
        np.arange(X.shape[0]), X, y_magic, train_size=0.8
    )
    x_scaler = StandardFlexibleScaler(column_wise=False).fit(X)
    # Center total dataset
    X = x_scaler.transform(X)
    # Center training data
    X_train = x_scaler.transform(X_train)
    # Center training data
    X_test = x_scaler.transform(X_test)

    clf = RandomForestClassifier(n_estimators=100, random_state=2)
    clf.fit(X_train, y_train)
    clf.predict(X_test)
    print("Accuracy on test set:" + str(clf.score(X_test, y_test)))
    p_random_for = clf.predict_proba(X)

    y_tot = p_random_for[:, 1].reshape(-1, 1)
    i_train, i_test, X_train, X_test, y_train, y_test = train_test_split(
        np.arange(X.shape[0]), X, y_tot, train_size=0.8
    )

    x_scaler = StandardFlexibleScaler(column_wise=False).fit(X)
    y_scaler = StandardFlexibleScaler(column_wise=True).fit(y_tot)

    # Center total dataset
    X = x_scaler.transform(X)
    Y = y_scaler.transform(y_tot)

    # Center training data
    X_train = x_scaler.transform(X_train)
    y_train = y_scaler.transform(y_train)

    # Center training data
    X_test = x_scaler.transform(X_test)
    y_test = y_scaler.transform(y_test)

    pcovr = PCovR(mixing=mixing, n_components=n_PC)
    pcovr.fit(X, Y)
    T = pcovr.transform(X)
    yp = pcovr.predict(X)
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    ax1.scatter(T[:, 0], T[:, 1], s=50, alpha=0.3, c=p_random_for[:, 1], cmap="bwr")
    ax1.set_xlabel(r"$PCov_1$")
    ax1.set_ylabel(r"$PCov_2$")

    ax2.scatter(Y, yp, s=50, alpha=0.5, c=np.abs(Y - yp), cmap="bone_r")
    ax2.plot(ax2.get_xlim(), ax2.get_xlim(), "r--")
    ax2.set_xlabel(r"y")
    ax2.set_ylabel(r"predicted y")
    sns.despine(left=True, bottom=True)

    #     #Exploring correlation with other features
    #     if DATA_MP.geo.exists():

    #         upper_bound=0.4 #number above which the correlation is considered HIGH (maximum is 1)
    #         lower_bound=0 #number below which the correlation is considered LOW (minimumis -1)

    #         col=range(0, len(X_raw))
    #         df = pd.DataFrame(data=X_raw, index=col)
    #         df[r'PC$_{1}$']=T[:,0]
    #         corr=df.corrwith(df[r'PC$_{1}$'])
    #         print(min(corr), max(corr), len(corr))
    #         print(corr)
    #         low_corr_soap=[]
    #         high_corr_soap=[]
    #         for i in range(len(corr)-1):
    #             if corr[i] <- lower_bound:
    #                 low_corr_soap.append(i)
    #             if corr[i] > upper_bound:
    #                 high_corr_soap.append(i)
    #             high_corr=[coeff[i] for i in high_corr_soap]
    #             low_corr=[coeff[i] for i in low_corr_soap]
    #         npzfile = np.load(DATA_MP.geo, allow_pickle=True)
    #         magic=npzfile['magic']
    #         natoms=npzfile['natoms']
    #         std_ratio=npzfile['std_ratio']
    #         n_species=npzfile['n_species']
    #         alpha=npzfile['alpha']
    #         x=npzfile['x']
    #         cn=npzfile['cn']
    #         packing=npzfile['packing']
    #         fcc=npzfile['fcc']
    #         hcp=npzfile['hcp']

    #         fig, axes = plt.subplots(figsize=(12,8))
    #         dict_corr={r'$\alpha$':alpha,r'$x$':x, r'$\sigma_{radii}$':std_ratio,
    #             r'$CN_{avg}$':cn, r'$PF$':packing,r'P$_{classif}$':p_random_for[:,1],
    #               r'$N_{species}$': n_species,r'PCov$_{1}$': T[:,0], r'PCov$_{4}$': T[:,3]}
    #         df_corr= pd.DataFrame(dict_corr)
    #         sns.heatmap(df_corr.corr(), vmin=-1, vmax=1, annot=True)
    #         plt.show()

    if not DATA_MP.chem_red.exists():
        data = write_input(
            path="../r4data/MP/chem_red.npz",
            meta={"name": "MP PCovR"},
            frames=my_frames,
            properties={
                **{
                    f"PCoV_{i+1}": {"target": "structure", "values": T[:, i]}
                    for i in range(n_PC)
                },
                "magic": {
                    "target": "structure",
                    "values": magic,
                },
                "magic probability": {
                    "target": "structure",
                    "values": p_random_for[:, 1],
                },
            },
        )